In [70]:
import pandas as pd
import jieba
corpus = pd.read_csv('./corpus.csv')
names = {'wrong_answer':0, 'exceed_expect':2, 'communication_skill':3, 'think_otherwise':4, 'shuffle':5, 'grasp_needs':6}
corpus

,id,matched_sentences,prob,target,name,matched_prob,keywords
0,2b755672-1c04-4574-b0aa-e445bca64841,不是这个意思\n我不是说这个\n你没理解我的意思\n听不懂我说的话吗\n不是这样子的\n你在...,0.8,all,答非所问,0.5,你没听明白\n你说的不对\n你不懂\n什么呀\n你说错了吧\n不是这样的\n你弄错了\n我是...
1,592cb7e5-26b3-4f94-8b9a-2bba0d952a3b,NaN,0.8,all,辱骂指责,0.5,无语\n变态\n傻子一样\n傻逼\n真烦\n傻缺\n啰嗦\n反应迟钝\n蠢货\n脑子进水了吧...
2,80d0f07a-875c-4853-b1ff-3f272ace845d,给我解决那个小女孩，应该表扬她\n你的服务我非常满意\n你服务非常好\n服务态度非常好\n你...,0.6,客户,用心超越期望,0.5,声音甜美\n你态度真好\n服务非常好\n声音好美啊\n服务挺好\n这服务好啊\n态度这么好\...
3,9291252b-4041-4d6b-b95c-682854db5990,我不是说了吗\n这样都听不懂吗\n我怎么说你才懂\n能不能别这么激动\n这可不是我说的\n没...,0.8,all,沟通技巧,0.5,这是你说的\n没办法\n解决不了\n只能这样了\n我也没办法\n你懂什么啊\n您这话说的\n...
4,9caf9b51-4fd9-4600-b877-f80c647e18bb,我这已经用好多年了，啥，额度就没调一次，没调一次额度?\n那我前两张下来就是白金，怎么到你们...,0.6,客户,换位思考,0.5,这么麻烦\n太难刷了\n跑太远了\n可打通了\n已下表反馈\n操你妈地\n那我不管\n你们搞...
5,ca6f955b-a5a5-4bee-83bf-c1be88957caa,你到时候再打电话把\n要转之前同事\n这个事情不是我跟进的\n那你去找别人啊\n又不是我弄的...,0.8,all,推诿,0.5,这么麻烦\n太难刷了\n跑太远了\n可打通了\n已下表反馈\n操你妈地\n那我不管\n你们搞...
6,cbcefb70-396c-4e72-ab40-bf98edfb97cb,没有明白你的意思\n是这样的吗？\n不知道你在说什么\n不知道你是什么意思\n再说什么乱七八...,0.8,all,把握需求,0.5,你说什么\n没听到\n你再说一遍\n听不懂\n什么鬼\n什么东西\n不太理解\n我不懂\n您...


In [67]:
# corpus['matched_sentences'][0].split('\n')
corpus_dic = {}
list1 = list(corpus['name'])
list2 = []
for i in range(7):
    if i != 1:
        list2.append(corpus['matched_sentences'][i].split('\n'))
    else:
        list2.append(corpus['keywords'][1].split('\n'))
for j in range(7):
    corpus_dic[list1[j]] = list2[j]

In [68]:
import json
with open('./corpus_original.json', 'w') as f:
    json.dump(corpus_dic, f, ensure_ascii = False)

In [2]:
result = {}
for name in names:
    result[name] = ''
    words = jieba.cut(corpus['matched_sentences'][names[name]].replace('\n', '').replace('，', '').replace('。', ''))
    for word in words:
        result[name] = result[name] + ' ' + word 

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.490 seconds.
Prefix dict has been built succesfully.


In [6]:
list1 = []
list2 = []
for name in names:
    list1.append(name)
    list2.append(result[name])
df_result = {0:list1, 1:list2}
df = pd.DataFrame(df_result)
df.to_csv('./cut_result_corpus.csv')

In [26]:
a = pd.read_csv('./cut_result_corpus.csv')

In [31]:
a[str(1)]

0     不是 这个 意思 我 不是 说 这个 你 没 理解 我 的 意思 听不懂 我 说 的话 吗...
1     给 我 解决 那个 小女孩 应该 表扬 她 你 的 服务 我 非常 满意 你 服务 非常 ...
2     我 不是 说 了 吗 这样 都 听不懂 吗 我 怎么 说 你 才 懂 能 不能 别 这么 ...
3     我 这 已经 用 好多年 了 啥 额度 就 没调 一次 没调 一次 额度 ? 那 我 前 ...
4     你 到时候 再 打电话 把 要 转 之前 同事 这个 事情 不是 我 跟进 的 那 你 去...
5     没有 明白 你 的 意思 是 这样 的 吗 ？ 不 知道 你 在 说 什么 不 知道 你 ...
Name: 1, dtype: object

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
class data():
    def __init__(self):
        self.texts = list(pd.read_csv('./cut_result_corpus.csv')[str(1)])

    def get_bag_of_words(self):
        corpus = self.texts
        vectorizer = CountVectorizer()
        vectorizer.fit(corpus)
        return vectorizer, vectorizer.transform(corpus)

    def get_tfidf_dict(self):
        dic = {}
        vectorizer, X = self.get_bag_of_words()
        tfidf_transformer = TfidfTransformer()
        tfidf_transformer.fit(X.toarray())
        for idx, word in enumerate(vectorizer.get_feature_names()):
            dic[word.encode('utf-8')] = tfidf_transformer.idf_[idx]
        tfidf = tfidf_transformer.transform(X)
        return dic, tfidf

data_util = data()
dic, tfidf = data_util.get_tfidf_dict()

In [39]:
list1 = []
list2 = []
for name in dic:
    list1.append(name.decode())
    list2.append(dic[name])
df_result = {0:list1, 1:list2}
df = pd.DataFrame(df_result)
df.to_csv('./word_tfidf.csv')

In [40]:
pd.read_csv('./word_tfidf.csv')

,Unnamed: 0,0,1
0,0,10,2.252763
1,1,1000,2.252763
2,2,15,2.252763
3,3,15000,2.252763
4,4,20,2.252763
5,5,2000,2.252763
6,6,24,2.252763
7,7,30,2.252763
8,8,315,2.252763
9,9,app,2.252763


In [41]:
corpus = pd.read_csv('./corpus.csv')

In [45]:
corpus['matched_sentences']

0    不是这个意思\n我不是说这个\n你没理解我的意思\n听不懂我说的话吗\n不是这样子的\n你在...
1                                                  NaN
2    给我解决那个小女孩，应该表扬她\n你的服务我非常满意\n你服务非常好\n服务态度非常好\n你...
3    我不是说了吗\n这样都听不懂吗\n我怎么说你才懂\n能不能别这么激动\n这可不是我说的\n没...
4    我这已经用好多年了，啥，额度就没调一次，没调一次额度?\n那我前两张下来就是白金，怎么到你们...
5    你到时候再打电话把\n要转之前同事\n这个事情不是我跟进的\n那你去找别人啊\n又不是我弄的...
6    没有明白你的意思\n是这样的吗？\n不知道你在说什么\n不知道你是什么意思\n再说什么乱七八...
Name: matched_sentences, dtype: object